# Tutorial BCI Carga Datos Riesgo Crediticio

Este tutorial muestra buenas practicas de carga de datos.

Muestra carga de carpetas con archivos raw (ejemplo CSV) y despues su posterior traspaso a una tabla parquet en la base de datos que apunta a ADLS.

![carga](files/images_doc/dbricks_cargaRaw.png)

Tambien muestra como pasar de unaa tabla Parquet a una tabla Delta (soporta ACID)

Previamente se debio haber cargado archivos con opción Databricks, esto es opción de menu "Data", botón "add data"

Luego opción "Upload File" y cargar en en path

> /FileStore/tables/rawdbfs/

![uploadfiles](files/images_doc/dbricks_UploadFiles.png)

Para subir carpetas se recomienda arrastrar la carpeta sobre vtna Databricks

</br>
>* nota importante: la data siempre debe quedar finalmente en ADLS (Storage), y por lo tanto las tablas de datos siempre deben apuntar al montaje ADLS.
</br>
</br>
> luego esta carga de archivos en DBFS es solo transitoria, una vez traspasados a la tabla destino, estos archivos deben ser eliminados.

Una vez subido los archivos, vamos a tomarlos desde este notebook y vamos a generar o cargar la tabla respectiva.

Vamos a verificar que estan los archivos que hicimos upload

In [0]:
%fs ls /FileStore/tables/rawdbfs/

Tambien lo podemos listar con Python y la libreria de databricks dbutils

In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/tables/rawdbfs/"))

Vamos a ver las bases de datos que estamos manejando en Databricks, se pueden ver tambien con la opción de menu "Data".

In [0]:
%sql
SHOW DATABASES

Esta base de datos fue creada apuntando a un path del Storage ADLS (location)

Ahora toda tabla que creemos en esta base de datos va a estar en ADLS por defecto

In [0]:
%sql 
DESCRIBE DATABASE risgcred_db

Vamos a crear una tabla Parquet a partir de una carga de archivos Raw.

Creo dataframe leyendo de la carpeta RAW en formato csv, infiero schema, para luego escribirlo como tabla (parquet)

In [0]:
dfPath = "dbfs:/FileStore/tables/rawdbfs/tmpmargen_jul2020/"
dfCSV=spark.read.format("csv").option("header","true")\
  .option("inferSchema", "true").load(dfPath)

In [0]:
dfCSV.show()

Para efectos de la demo vamos a borrar la tabla

In [0]:
%sql
DROP TABLE IF EXISTS risgcred_db.tmp_margen_acme_tb

Solo la primera carga se crea la tabla.

Al guardarlo como tabla e indicar la base de datos ("brujulafinanciera_db."), entonces automaticamente lo guarda en ADLS, porque saca la ruta ADLS de la base de datos (location). Si no se indica nada entonces crea la tabla como parquet por defecto.

In [0]:
dfCSV.write.saveAsTable("risgcred_db.tmp_margen_acme_tb")

In [0]:
%sql SELECT * FROM risgcred_db.tmp_margen_acme_tb

In [0]:
%sql
SELECT COUNT(*) FROM risgcred_db.tmp_margen_acme_tb

Podemos ver la estructura con que se creo la tabla. Aqui no aparece location (hacia ADLS) porque lo hereda de la base de datos por defecto.

In [0]:
%sql
SHOW CREATE TABLE risgcred_db.tmp_margen_acme_tb

Ahora aparece la tabla como subcarpeta en el montaje (que corresponde a ADLS)

In [0]:
%fs ls /mnt/bcirisgcredcontain/risgcred_db/

Y podemos verificar que esta en formato parquet, revisando los archivos que se crearon.

In [0]:
%fs ls /mnt/bcirisgcredcontain/risgcred_db/tmp_margen_acme_tb/

Con esto comprobamos que ya los archivos fueron cargados como datos en la nueva tabla

Para mantener el orden, vamos a borrar los archivos cargados, quedando limpia la carpeta RAW

In [0]:
%fs rm -r /FileStore/tables/rawdbfs/tmpmargen_jul2020/

Como es una tabla ya podemos incluso hacer INSERT con SQL

In [0]:
%sql

INSERT INTO risgcred_db.tmp_margen_acme_tb VALUES (19700328,28,"Alfa","XXX","CLASE A","28X","CUENTA Y","CREDITOS ACME","ZZ PLAZO","PRODUCTO Z",1234567890,"TRANSACCION ALFA",0.0000,0.0000,28.28);

Refresh es importante sobre todo si hay otro procesos afuera agregando archivos o insertando datos, para tener data actualizada.

In [0]:
%sql
REFRESH TABLE risgcred_db.tmp_margen_acme_tb

In [0]:
%sql
SELECT * FROM risgcred_db.tmp_margen_acme_tb WHERE Id_Periodo = 19700328

In [0]:
%sql
SELECT COUNT(*) FROM risgcred_db.tmp_margen_acme_tb 

En las siguientes cargas se agrega la data solamente, ya no se crea la tabla.

In [0]:
dfPathAgo = "dbfs:/FileStore/tables/rawdbfs/tmpmargen_ago2020/"
dfSig=spark.read.format("csv").option("header","true")\
  .option("inferSchema", "true").load(dfPathAgo)

In [0]:
dfSig.show()

El write es del tipo append (agrega nueva data a la tabla)

In [0]:
dfSig.write.mode("append").saveAsTable("risgcred_db.tmp_margen_acme_tb")

In [0]:
%sql
SELECT COUNT(*) FROM risgcred_db.tmp_margen_acme_tb

Vamos a leer toda la data (parquet), y vamos a crear un tabla en delta.

hay varias formas de leer toda la data en DataFrame

ref https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#dataframe-faqs

In [0]:
dfAll01 = table("risgcred_db.tmp_margen_acme_tb")
dfAll01.count()


In [0]:
dfAll02 = spark.sql("select * from risgcred_db.tmp_margen_acme_tb")
dfAll02.count()

incluso podemos leer la data directo de la carpeta que corresponde a la tabla.

ref https://docs.databricks.com/data/data-sources/read-parquet.html

In [0]:
dfAll03=spark.read.parquet("dbfs:/mnt/bcirisgcredcontain/risgcred_db/tmp_margen_acme_tb/")
dfAll03.count()

Ahora creamos la tabla delta (termina en "_dtb") destino a partir del dataframe

ref https://docs.databricks.com/_static/notebooks/delta/optimize-scala.html

In [0]:
%sql
DROP TABLE IF EXISTS risgcred_db.tmp_margen_delta_dtb

In [0]:
dfAll01.write.format("delta").saveAsTable("risgcred_db.tmp_margen_delta_dtb")

In [0]:
%sql
SELECT * FROM risgcred_db.tmp_margen_delta_dtb

podemos revisar que la creo en ADLS montado

In [0]:
%fs ls /mnt/bcirisgcredcontain/risgcred_db/tmp_margen_delta_dtb/